In [1]:
%pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\KAIRAV\TensorForce\TensorForce\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from a specific .env file
load_dotenv(dotenv_path=r'C:\Users\KAIRAV\TensorForce\ML\model2\api_kairav.env')

# Access the environment variables
ALPACA_API_KEY = os.getenv('TRADING_CLIENT_KEY')
ALPACA_SECRET_KEY = os.getenv('TRADING_CLIENT_SECRET')


In [8]:
%pip install fastapi

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\KAIRAV\TensorForce\TensorForce\Scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
import numpy as np
import pandas as pd
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from keras.models import load_model

# Manually specify the full path to the model
model_path = r'C:\Users\KAIRAV\TensorForce\ML\model1\model_99.h5'

# Load LSTM model
model = load_model(model_path)

# Alpaca API placeholder (replace with actual keys)
ALPACA_API_KEY = "your_alpaca_api_key"
ALPACA_SECRET_KEY = "your_alpaca_secret_key"

# FastAPI setup
app = FastAPI()

# Define input schema
class PredictionRequest(BaseModel):
    open: float
    high: float
    low: float
    prev_close: float
    volume: int
    vwap: float
    close: float  # Actual Close (for reference)

# Define threshold values
THRESHOLD_BUY = 1.0  # Percentage
THRESHOLD_SELL = 1.0  # Percentage

# Buy/Sell Signal Logic
def generate_signal(data, predicted_close):
    """
    Generates buy/sell signal based on thresholds and indicators.
    """
    current_close = data["close"]
    change_percent = ((predicted_close - current_close) / current_close) * 100

    # Threshold logic
    buy_signal = change_percent > THRESHOLD_BUY
    sell_signal = change_percent < -THRESHOLD_SELL

    # Calculate RSI (Relative Strength Index)
    delta = np.diff(data["close_prices"])
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.mean(gain[-14:])
    avg_loss = np.mean(loss[-14:])
    rs = avg_gain / avg_loss if avg_loss != 0 else 0
    rsi = 100 - (100 / (1 + rs))

    # Calculate Moving Averages
    sma = np.mean(data["close_prices"][-14:])
    ema = np.mean(data["close_prices"][-14:])  # For simplicity, EMA as SMA here

    # MACD Calculation (Fast - Slow)
    short_window = 12
    long_window = 26
    signal_window = 9

    macd = np.mean(data["close_prices"][-short_window:]) - np.mean(data["close_prices"][-long_window:])
    macd_signal = np.mean(data["close_prices"][-signal_window:])

    macd_cross = macd > macd_signal  # Check if MACD > Signal Line

    # Combine Threshold with Indicators
    buy_signal = buy_signal and (rsi < 30 or current_close > sma)
    sell_signal = sell_signal and (rsi > 70 or current_close < sma)

    return buy_signal, sell_signal, {"rsi": rsi, "sma": sma, "ema": ema, "macd": macd, "macd_signal": macd_signal}

@app.post("/predict")
def predict(request: PredictionRequest):
    """
    Predicts the closing price and generates buy/sell signals.
    """
    # Prepare data for model
    input_features = np.array([[request.open, request.high, request.low, request.prev_close, request.volume, request.vwap]])
    input_features = np.reshape(input_features, (input_features.shape[0], 1, input_features.shape[1]))

    # Model prediction
    predicted_close = model.predict(input_features)[0][0]

    # Historical close prices for indicators (mocked for simplicity)
    historical_data = {"close_prices": np.random.uniform(100, 200, 100)}  # Replace with actual historical data

    # Generate buy/sell signals
    buy_signal, sell_signal, indicators = generate_signal({"close": request.close, **historical_data}, predicted_close)

    return {
        "predicted_close": predicted_close,
        "buy_signal": buy_signal,
        "sell_signal": sell_signal,
        "indicators": indicators
    }

# Integration with Alpaca API (Placeholder)
def place_order(signal_type: str, quantity: int, symbol: str):
    """
    Places an order using Alpaca API based on the signal.
    """
    import requests

    headers = {
        "APCA-API-KEY-ID": ALPACA_API_KEY,
        "APCA-API-SECRET-KEY": ALPACA_SECRET_KEY
    }

    url = f"https://paper-api.alpaca.markets/v2/orders"
    payload = {
        "symbol": symbol,
        "qty": quantity,
        "side": signal_type,  # "buy" or "sell"
        "type": "market",
        "time_in_force": "gtc"  # Good 'til canceled
    }

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        return {"message": f"{signal_type.capitalize()} order placed successfully!", "details": response.json()}
    else:
        raise HTTPException(status_code=response.status_code, detail=response.json())

@app.post("/trade")
def trade(request: PredictionRequest):
    """
    Predicts the closing price and automatically places buy/sell orders.
    """
    prediction_response = predict(request)
    symbol = "AAPL"  # Replace with the actual stock symbol
    quantity = 1  # Replace with desired quantity

    if prediction_response["buy_signal"]:
        return place_order("buy", quantity, symbol)
    elif prediction_response["sell_signal"]:
        return place_order("sell", quantity, symbol)
    else:
        return {"message": "No significant signal to act on."}
